# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

In [69]:
data= pd.read_csv("../data/trip_duration_task.csv")
data.drop(['id','pickup_datetime', 'dropoff_datetime', 'vendor_id', 'passenger_count' ], axis=1, inplace=True)
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,-73.953918,40.778873,-73.963875,40.771164,400
1,-73.988312,40.731743,-73.994751,40.694931,1100
2,-73.997314,40.721458,-73.948029,40.774918,1635
3,-73.961670,40.759720,-73.956779,40.780628,1141
4,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...
729317,-73.965919,40.789780,-73.952637,40.789181,296
729318,-73.996666,40.737434,-74.001320,40.731911,315
729319,-73.997849,40.761696,-74.001488,40.741207,673
729320,-74.006706,40.708244,-74.013550,40.713814,447


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727475 entries, 0 to 729321
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pickup_longitude   727475 non-null  float64
 1   pickup_latitude    727475 non-null  float64
 2   dropoff_longitude  727475 non-null  float64
 3   dropoff_latitude   727475 non-null  float64
 4   trip_duration      727475 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 33.3 MB


Предобработка данных

1) Убрал строки с null-value в столбце "vendor_id"
2) Заполнил строки с null-value в столбце "pickup_latitude"

In [71]:
y = data['trip_duration']
X = data.drop(['trip_duration'], axis=1)


In [72]:
y

0          400
1         1100
2         1635
3         1141
4          848
          ... 
729317     296
729318     315
729319     673
729320     447
729321    1224
Name: trip_duration, Length: 727475, dtype: int64

In [73]:
X

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,-73.953918,40.778873,-73.963875,40.771164
1,-73.988312,40.731743,-73.994751,40.694931
2,-73.997314,40.721458,-73.948029,40.774918
3,-73.961670,40.759720,-73.956779,40.780628
4,-74.017120,40.708469,-73.988182,40.740631
...,...,...,...,...
729317,-73.965919,40.789780,-73.952637,40.789181
729318,-73.996666,40.737434,-74.001320,40.731911
729319,-73.997849,40.761696,-74.001488,40.741207
729320,-74.006706,40.708244,-74.013550,40.713814


In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [75]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((581980, 4), (581980,), (145495, 4), (145495,))

In [76]:
lr = LinearRegression().fit(X_train, y_train)

In [77]:
lr.predict(X_test)
list_column_weight = list(zip(X.columns,lr.coef_))
list_column_weight


[('pickup_longitude', 2455.9534529464),
 ('pickup_latitude', -3625.4908200280233),
 ('dropoff_longitude', -643.2648930974929),
 ('dropoff_latitude', -2224.3409323351966)]

In [78]:
y_pred = lr.predict(X_test)
y_pred.shape

(145495,)

In [79]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')
print(f'R^2: {lr.score(X, y)}')

MAE: 596.6168790600725
MSE: 10159688.2470863
RMSE: 3187.4265869328347
MAPE: 1.2632660103435234
R^2: 0.003915683354134947
R^2: 0.0030848870752319124


In [80]:
len(lr.coef_)
lr.coef_

array([ 2455.95345295, -3625.49082003,  -643.2648931 , -2224.34093234])

In [81]:
ridge = Ridge(alpha=0.9).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 596.6211172615192
MSE: 10159672.234594177
RMSE: 3187.4240751105235
MAPE: 1.263254101683235
R^2: 0.003917253263750919


array([ 2453.8365091 , -3621.1621662 ,  -641.68008219, -2223.30942753])

In [82]:
parameters = {'alpha': np.arange(0, 1, 0.1)}
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.9}

In [83]:
lasso = Lasso(alpha=475)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 621.3435482093176
MSE: 10199632.122365268
RMSE: 3193.6862905372013
MAPE: 1.2565324939179645
R^2: 0.003917253263750919


array([ 2453.8365091 , -3621.1621662 ,  -641.68008219, -2223.30942753])

In [96]:
polynomialFeatures = PolynomialFeatures(2)
X = X.values.reshape(-1, 4)
y = y.values.reshape(-1, 1)


In [117]:
lr = LinearRegression().fit(X, y)
lr.coef_

array([[ 2538.76248118, -3550.59502695,  -488.59114279, -1890.35671589]])

In [118]:
X_pf = polynomialFeatures.fit_transform(X)